In [122]:
import sys, os, glob
user = os.path.expanduser('~')
sys.path.insert(0, user + '/Dropbox/Projects/Code/utilities')
sys.path.insert(0, user + '/Dropbox/Projects/Code/snt_behavior/preprocessing')
sys.path.insert(0, '..') 

from project_utils import *
from snt_info import *
from parsing_functions import * 
from preprocess_snt import preprocess_snt
from standard_modules import *
from circ_stats import *
import csv
from functools import reduce

char_roles = ['first', 'second', 'assistant', 'powerful', 'boss', 'neutral']
ver_skincolor = [['Brown','Brown','Brown','White','White','Brown'],
                 ['White','White','White','Brown','Brown','White'],
                 ['Brown','Brown','Brown','White','White','White'],
                 ['White','White','White','Brown','Brown','Brown']]
ver_gender = [['Woman','Man','Man','Woman','Man','Woman'],
              ['Woman','Man','Man','Woman','Man','Woman'],
              ['Man','Woman','Woman','Man','Woman','Man'],
              ['Man','Woman','Woman','Man','Woman','Man']]

# names frm dots
ver_name = [['Olivia','Peter','Anthony','Newcomb','Hayworth','Kayce'],
            ['Olivia','Peter','Anthony','Newcomb','Hayworth','Kayce'],
            ['Peter','Olivia','Kayce','Newcomb','Hayworth','Anthony'],
            ['Peter','Olivia','Kayce','Newcomb','Hayworth','Anthony']]

syn_dir     = '/Volumes/synapse/projects/SocialSpace/Projects/SNT-fmri_CUD'
data_dir    = f'{syn_dir}/Data'
beh_dir     = f'{data_dir}/Behavior'
summary_dir = f'{data_dir}/Summary'

# SNT

In [16]:
# parse the logs
logs = glob.glob(f'{data_dir}/Behavior/Logs/*.log')
print(f'Found {len(logs)} logs')
xlsx = [f for f in glob.glob(f'{beh_dir}/Organized/*.xlsx') if '~$' not in f]
if len(logs) == len(xlsx):
    print('All logs appear to be processed')
else:
    for f in files:
        if not os.path.exists(f):
            parse_log(f, exp='KB', fmri=True)
            
# compute the behavioral geometry
if os.path.exists(f'{summary_dir}/SNT-task_n{len(xlsx)}.xlsx'):
    print('Behavioral preprocessing is completed')
else:
    xlsx_files = glob.glob(beh_dir + '/Organized/*.xlsx')
    preprocess_snt(beh_dir, xlsx_files)

Found 80 logs
All logs appear to be processed
Behavioral preprocessing is completed


# Questionnaires

In [32]:
# sr_df = pd.read_csv(glob.glob(data_dir + '/Questionnaires/CURRENT*')[0])

# # exclude cols (phi, timestamps, others)
# phi = ['name','phi','phonenum', 'timestamp', 'complet']
# cols = sr_df.columns
# cols_incl = list(cols[[not any(phi in c for phi in phi) for c in cols]])
# sr_df = sr_df[cols_incl]

# # sub ids
# sub_list = [sub.replace('P','') for sub in sr_df['record_id'].values]
# sub_list[np.where(np.array(sub_list)=='21002_20013')[0][0]] = '21002'
# sr_df['record_id'] = sub_list
# sr_df.rename(columns={'record_id':'sub_id'}, inplace=True)

# # sni 
# num_ppl = ((sr_df['sni_marital_status'] == 1) * 1).values
# network_div = ((sr_df['sni_marital_status'] == 1) * 1).values
# other_sni = sr_df[['sni_numbr_children_contact','sni_relatives_talk_to_biweekly',
#                     'sni_friends_contact_biweekly','sni_religious_affiliation_contact_biweekly',
#                     'sni_students_teachers_biweekly_contact','sni_coworkers_biweekly_contact',
#                     'sni_neighboors_biweekly_contact','sni_volunteers_biweekly_contact',
#                     'sni_other_volunteer_grp_member_biweekly_contact','sni_total_nmbr_other_volunteers_biweekly_contact']]

# num_ppl = num_ppl + np.sum(np.nan_to_num(other_sni.values), 1)
# network_div = network_div + np.sum((np.nan_to_num(other_sni.values) > 0) * 1, 1)

# parents_sni = sr_df[['sni_in_laws_talk_biweekly','sni_parents_contact_biweekly']]
# both = (parents_sni == 'both') * 2
# mother = (parents_sni == 'mother') * 1
# father = (parents_sni == 'father') * 1

# sr_df['sni_num_ppl'] = num_ppl + np.sum(both.values + mother.values + father.values, 1)
# sr_df['sni_network_div'] = network_div + np.sum(both.values>0, 1) + np.sum(mother.values>0, 1) + np.sum(father.values>0, 1)

# sr_df.to_excel(data_dir + '/Summary/Questionnaires_n' + str(len(sr_df)) + '.xlsx')

# Memory

In [18]:
mem_df = pd.read_excel(glob.glob(f'{data_dir}/Summary/SNT-memory*_raw.xlsx')[0])

FV = ['Newcomb','Newcomb','Hayworth','Kayce','Jessica','Anthony','Jessica','Chris',
      'Kayce','Newcomb','Hayworth','Jessica','Kayce','Anthony','Anthony','Kayce',
      'Chris','Newcomb','Anthony','Chris','Kayce','Chris','Newcomb',
      'Jessica','Hayworth','Jessica','Hayworth','Anthony','Chris', 'Hayworth']
MV = ['Newcomb','Newcomb','Hayworth','Anthony','Chris','Kayce','Chris','Jessica','Anthony','Newcomb',
      'Hayworth','Chris','Anthony','Kayce','Kayce','Anthony','Jessica','Newcomb','Kayce','Jessica',
      'Anthony','Jessica','Newcomb','Chris','Hayworth','Chris','Hayworth','Kayce','Jessica','Hayworth']
roles = ['powerful','powerful','boss','neutral','first','assistant','first','second','neutral','powerful',
         'boss','first','neutral','assistant','assistant','neutral','second','powerful','assistant','second',
         'neutral','second','powerful','first','boss','first','boss','assistant','second','boss']
ques = [f'memory_{r+1}_{role}' for r,role in enumerate(roles)]

dfs = []
for s,sub in mem_df.iterrows():
    v = sub.Task_ver
    if v in [1,2]: ans = np.array([x.lower() for x in FV])
    else:          ans = np.array([x.lower() for x in MV])
    resps = np.array([x.lower() for x in list(sub.values[2:])])
    correct = list((resps==ans) * 1)
    acc = np.mean(correct)
    
    df = pd.DataFrame([sub.Sub_id, v, acc] + correct).T
    df.columns = ['sub_id','task_version','memory_mean'] + ques
    dfs.append(df)
mem_df = pd.concat(dfs)
mem_df.to_excel(f'{data_dir}/Summary/SNT-memory_n{len(mem_df)}_processed.xlsx')
print('Memory processing completed')

Memory processing completed


# Dots

In [19]:
# parse the dots
dots_dir = f'{data_dir}/Dots'
jpgs = glob.glob(f'{dots_dir}/Dots*jpg')
print(f'Found {len(jpgs)} dots jpgs')
if os.path.exists(f'{data_dir}/Summary/SNT-dots_n{len(jpgs)}.xlsx'):
    print('Dots appear to be processed')
else:
    dfs = []
    for j, jpg in enumerate(jpgs):
        print(f'{j} {jpg}', end="\r")
        try:
            sub_id = jpg.split('Dots_')[1].split('.jpg')[0]
            df = define_char_coords(load_image(jpg))[1]
            df.insert(0, 'sub_id', sub_id)
            dfs.append(df)
        except:
            print(f'ERROR: {fname}')

    coords_df = pd.concat(dfs)
    coords_df.to_excel(f'{data_dir}/Summary/SNT-dots_n{len(coords_df)}.xlsx', index=False)    

Found 80 dots jpgs
Dots appear to be processed


# fMRI QC

In [80]:
tsv_fnames = glob.glob(f'{syn_dir}/QC/cfd_files/*.tsv')
tsv_fnames.sort()

outname    = f'{summary_dir}/SNT-fMRI_QC_n{len(tsv_fnames)}.xlsx'
overwrite  = True
cfd_cols   = ['framewise_displacement', 'rmsd', 'std_dvars']
# -- dvars: rate of change of BOLD signal across the entire brain at each volume

if (os.path.exists(outname)) & (not overwrite):
    print('fMRI QC is already summarized')
else:
    print(f'Found {len(tsv_fnames)} confound files')
    dfs = []
    for i, fname in enumerate(tsv_fnames):
        
        print(i+1, end="\r")
        
        with open(fname) as f: 
            cfds = [r for r in csv.reader(f, delimiter="\t", quotechar='"')] # load
        
        sub_id = int(fname.split('sub-P')[-1].split('_')[0])
        cfd_df = pd.DataFrame(cfds[1:], columns=cfds[0])
        cfd_df.replace(to_replace='n/a', value=np.nan, inplace=True)
        cfd_df = cfd_df.astype(float)  
        
        ## mean and max of confounds
        means  = np.nanmean(cfd_df.loc[:, cfd_cols], 0)
        maxes  = np.nanmax(cfd_df.loc[:, cfd_cols], 0)

        ## find outlier volumes
        # framewise displacement > 1 voxel (2.1mm)
        fd_onevox   = cfd_df['framewise_displacement'] > 2.1
        fd_onevox_n = np.sum(fd_onevox) 
        if fd_onevox_n>0:
            print(sub_id)
        try:  
            fd_onevox_vols = list(np.where(fd_onevox==1)[0])
        except:                
            fd_onevox_vols = [np.nan] # none

        # fmripreps outliers: motion or intensity spikes
        # -- defaults are FD > 0.5 mm or DVARS > 1.5 (seems fairly conservative)
        # -- these columns could be used as regressors in first level model
        try: 
            cols = [c for c in cfd_df.columns if 'motion_outlier' in c]
            outlier_vols = [np.where(cfd_df[c]==1)[0][0] for c in cols]
        except: 
            outlier_vols = [np.nan] # none 

        # % of decision volumes that are outliers
        outlier_decision   = np.round(len([v for v in outlier_vols if v in decision_epochs])/len(decision_epochs), 2) 
        fd_onevox_decision = np.round(len([v for v in fd_onevox_vols if v in decision_epochs])/len(decision_epochs), 2) 

        dfs.append(pd.DataFrame([sub_id] + list(means) + list(maxes) + [outlier_decision, fd_onevox_decision]).T)
    
df = pd.concat(dfs)
df.columns = ['sub_id','fd_mean','rmsd_mean','std_dvars_mean','fd_max','rmsd_max','std_dvars_max','fmriprep_outlier_proportion','fd_outlier_proportion']
df = df[['sub_id','fd_mean','fd_max','rmsd_mean','rmsd_max','std_dvars_mean','std_dvars_max','fmriprep_outlier_proportion','fd_outlier_proportion']] # reorganize
df.to_excel(outname, index=False)

Found 75 confound files
18004
18006
18007
18010
18015
18017
19004
19005
19007
19009
19012
19014
19016
19017
19025
19027
19028
19032
19042
19045
19051
19052
19053
19056
19057
19059
20001
20006
20007
20010
21002
21004
21013
21014
21020
21021
22001
22002


# Combine

In [123]:
dfs = []

# combine data
for fname in ['SNT-task_n*', 'SNT-memory*processed*',
              'SNT-fMRI_QC*', 'Questionnaires_*', 'SNT-task_versions.xlsx']:
    df = pd.read_excel(glob.glob(f'{summary_dir}/{fname}')[0])
    df.sort_values(by='sub_id', inplace=True)
    df['sub_id'] = df['sub_id'].astype(int)
    dfs.append(df)
summary_df = reduce(lambda x, y: pd.merge(x, y, on = 'sub_id'), dfs)
first_cols = ['sub_id', 'dx']
summary_df = summary_df[first_cols + [c for c in summary_df if c not in first_cols]]
del summary_df['version']

# task version info
dots_df = pd.read_excel(glob.glob(f'{summary_dir}/SNT-dots*')[0])
dots_df = summary_df[['sub_id', 'task_version']].merge(dots_df, on='sub_id')

sub_dfs = []
for s, sub in dots_df.iterrows():
    
    v = sub.task_version.astype(int) - 1
    for n, name in enumerate(ver_name[v]):
        sub.rename(index={name + '_affil':'dots_affil_' + char_roles[n]}, inplace=True)
        sub.rename(index={name + '_power':'dots_power_' + char_roles[n]}, inplace=True)
    gender   = pd.DataFrame(np.array(ver_gender[v]).reshape(1,-1),    columns=[c + '_gender' for c in char_roles])
    skintone = pd.DataFrame(np.array(ver_skincolor[v]).reshape(1,-1), columns=[c + '_skincolor' for c in char_roles])
    sub_dfs.append(sub.to_frame().T)
    
summary_df = summary_df.merge(pd.concat(sub_dfs), on=['sub_id','task_version'])
summary_df.to_excel(summary_dir + '/All-data_summary_n'+str(len(summary_df)) + '.xlsx', index=False)

In [120]:
# summarize all quality control
info = pd.read_excel(f'{syn_dir}/participants_info.xlsx')
qc_cols = ['sub_id', 'dx', 'memory_mean', 'missed_trials', 'rt_mean','fd_mean','fd_max',
           'rmsd_mean','rmsd_max','std_dvars_mean','std_dvars_max', 'fmriprep_outlier_proportion','fd_outlier_proportion']
qc_df = summary_df[qc_cols]

qc_df.insert(3, 'memory_incl', (qc_df['memory_mean'] > 0.20) * 1)
qc_df.insert(5, 'missed_trials_incl', (qc_df['missed_trials'] < 10) * 1)
qc_df.insert(15, 'fd_incl', (qc_df['fd_outlier_proportion'] < 0.05) * 1)
qc_df = info.merge(qc_df, on=['sub_id', 'dx'], how='outer')
inclusion = qc_df['other_incl'] * qc_df['memory_incl'] * qc_df['missed_trials_incl'] * qc_df['fd_incl']
qc_df.insert(0, 'inclusion', inclusion)
disp(qc_df.head(4))

qc_df.to_excel(f'{syn_dir}/participants_qc_n{len(qc_df)}.xlsx', index=False)

,inclusion,sub_id,dx,scan_date,xnat_id,notes,other_incl,memory_mean,memory_incl,missed_trials,...,rt_mean,fd_mean,fd_max,rmsd_mean,rmsd_max,std_dvars_mean,std_dvars_max,fmriprep_outlier_proportion,fd_outlier_proportion,fd_incl
0,1.0,18001,HC,NaT,NaN,NaN,1.0,0.833333,1.0,0.0,...,4.109746,0.147883,0.927599,0.085049,0.542953,1.006453,2.747418,0.04,0.00,1.0
1,1.0,18002,HC,NaT,NaN,NaN,1.0,0.333333,1.0,3.0,...,6.670429,0.156055,1.614437,0.099302,0.793133,1.010122,1.945367,0.02,0.00,1.0
2,1.0,18003,HC,NaT,NaN,NaN,1.0,0.466667,1.0,0.0,...,2.601905,0.197266,0.793762,0.128853,0.520482,1.075622,1.694489,0.01,0.00,1.0
3,1.0,18004,CD,NaT,NaN,NaN,1.0,0.266667,1.0,1.0,...,2.002444,0.462666,6.367780,0.275860,3.236020,1.075255,4.718619,0.38,0.02,1.0
